### Description

This script compares cest values of regions within network vs between network.


### Import Packages

In [6]:
import os
import glob
import numpy as np
import pandas as pd
#import network_fcon as fc
import scipy as sp
from scipy.stats import pearsonr
from scipy.stats import linregress
import seaborn as sns
import matplotlib.pyplot as plt
import re
import seaborn as sns
import statsmodels.formula.api as smf
from sklearn.decomposition import NMF

### Set Variables and Paths

In [11]:
# Set Variables
fieldstrength = '3T'
atlas = 'S1000'
bblids = []
sesids = []
nmaps = ["NMDA", "mGluR5", "GABA","D2"]
diag_scores = ["dx_pscat", "hstatus","sex", "age", "race","ethnic"] # ,"axis1_desc1","axis1_stat1"
diag_details = ["axis1_desc1", "axis1_desc2", "axis1_desc3","axis1_desc4","axis1_desc5", "axis1_desc6"]
#subjlist = subjlist[subjlist['fieldstrength'] == fieldstrength]
#subjs = subjlist['BBLID']

# Set paths
inpath = '/Users/pecsok/projects/GluCEST-fMRI/glucest-rsfmri/fmri_pipeline/parcellated_pipeline/'
outpath = '/Users/pecsok/projects/GluCEST-fMRI/glucest-rsfmri/fmri_pipeline/parcellated_pipeline/figures/' + atlas
print(outpath)
# Import group dataframes and set indices

#subjlist = pd.read_csv(path + "data/pipeline_input_all_subj.csv", sep=',') 
#cestmat = pd.read_csv("cest_parcelmat" + fieldstrength + atlas + ".csv", sep=',') 
cestmat = pd.read_csv(inpath + "filtered_cest_parcelmat" + fieldstrength + atlas + ".csv", sep=',') 
#fcmat = pd.read_csv(inpath + "fc_parcelmat_" + fieldstrength + atlas + ".csv", sep=',')
#rehomat = pd.read_csv(inpath + "reho_parcelmat" + fieldstrength + atlas + ".csv", sep=',')
grp_df = pd.read_csv(inpath + "filtered_grp_df" + fieldstrength + atlas + ".csv", sep=',')

# Reformat some dfs
cestmat.set_index('BBLID', inplace = True)
#fcmat.set_index('BBLID', inplace = True)
#rehomat.set_index('BBLID', inplace = True)
grp_df.set_index('BBLID', inplace = True)


/Users/pecsok/projects/GluCEST-fMRI/glucest-rsfmri/fmri_pipeline/parcellated_pipeline/figures/S1000


### Create functions

In [13]:
def filter(df, filter_list):
    filtered_columns = [col for col in df.columns if not any(substring in col for substring in filter_list)]
    return df[filtered_columns]

def keep(df, keep_list):
    keep_columns = [col for col in df.columns if any(substring in col for substring in keep_list)]
    return df[keep_columns]   

def corr_sig(df=None):
    p_matrix = np.zeros(shape=(df.shape[1],df.shape[1]))
    for col in df.columns:
        for col2 in df.drop(col,axis=1).columns:
            valid_data = df[[col,col2]].dropna()
            if not valid_data.empty:
                _ , p = pearsonr(valid_data[col],valid_data[col2])
                p_matrix[df.columns.to_list().index(col),df.columns.to_list().index(col2)] = p
            else:
                p_matrix[df.columns.to_list().index(col),df.columns.to_list().index(col2)] = np.nan
    return p_matrix



### Create correlation matrix and heatmap

In [12]:
# Filter df to exclude subj info
#subj_info = ["BBLID"] + ["Session"] + diag_scores + ["count"]
#heat_df = filter(grp_df, subj_info)
print(cestmat)

        Unnamed: 0  Session dx_pscat  hstatus      sex      age     race  \
BBLID                                                                      
20303            0    12276  noDSMdx       NC      1.0     19.2      1.0   
90217            1    12064      pro      PRO      2.0     25.0      1.0   
88608            2  motive2      psy      PRO      1.0     25.4      2.0   
21874            3    12043      NaN      PSY      2.0     24.1      1.0   
94288            4    11948  noDSMdx       NC      1.0     22.7      1.0   
...            ...      ...      ...      ...      ...      ...      ...   
135085          81  motive1      pro      PRO      2.0     18.9      1.0   
135277          82    12277      psy      PSY      2.0     30.2      2.0   
117397          83    10582    other     PROR      1.0     20.2      2.0   
87646           84    10496  Unknown  Unknown  Unknown  Unknown  Unknown   
93242           85    12525    other     PROR        1     25.3      1.0   

         et

In [23]:
# Assuming `df` is your data frame with brain regions and GluCEST values
# Transpose if necessary: df = df.T
pd.set_option('display.max_rows', None)

cestmeans = keep(cestmat, ["NZMean"])


total_columns = cestmeans.shape[1]
cestmeans = cestmeans.dropna(thresh=total_columns - 30)
nan_counts = cestmeans.isna().sum(axis=0)
print(nan_counts)

data_matrix = cestmeans.values


# Apply NMF
k = 7
nmf = NMF(n_components=k, init='random', random_state=42)
W = nmf.fit_transform(data_matrix)  # Basis matrix (regions)
H = nmf.components_  # Coefficients matrix (subjects)

# Optionally convert back to DataFrame for easier interpretation
basis_matrix = pd.DataFrame(W, columns=[f'Component_{i+1}' for i in range(k)])
coefficients_matrix = pd.DataFrame(H, index=[f'Subject_{i+1}' for i in range(H.shape[0])])

NZMean_502     9
NZMean_504    24
NZMean_518     6
NZMean_521     4
NZMean_532     5
NZMean_540    16
NZMean_542     1
NZMean_544     2
NZMean_545     3
NZMean_552     0
NZMean_555     6
NZMean_558     4
NZMean_560     1
NZMean_561     0
NZMean_562    27
NZMean_563     5
NZMean_564     0
NZMean_567     4
NZMean_574     0
NZMean_580     0
NZMean_581     3
NZMean_587     1
NZMean_590     1
NZMean_595     2
NZMean_596     1
NZMean_598     8
NZMean_606     2
NZMean_608     1
NZMean_613     4
NZMean_692     2
NZMean_707     1
NZMean_709     9
NZMean_712     9
NZMean_750     2
NZMean_751     1
NZMean_752     0
NZMean_754     3
NZMean_757     0
NZMean_759     0
NZMean_761     4
NZMean_762     2
NZMean_763     0
NZMean_764     0
NZMean_765     2
NZMean_766     0
NZMean_767     0
NZMean_768     1
NZMean_792     2
NZMean_793     0
NZMean_794     9
NZMean_795     2
NZMean_796     4
NZMean_797     4
NZMean_807     6
NZMean_809     2
NZMean_811     6
NZMean_812     2
NZMean_855     4
NZMean_856    

ValueError: Input X contains NaN.
NMF does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values